# Examen 1 - Optimización

## 1. 

Carlos debe limitarse a una dieta de carne de res y queso. A él se le restringe la cantidad de estos artículos a elegir ya que solo dispone de $2,450 para adquirirlos, pero es importante satisfacer ciertos requerimientos mínimos y minimizar el consumo de colesterol. La tabla siguiente refleja la cantidad (en miligramos) de vitamina A, B y C que contiene cada uno de los productos alimenticios, así como también su nivel (unidades) de colesterol. La tabla también incluye los requerimientos mínimos diarios de vitaminas y el costo de cada uno de los productos.

| Componente de los alimentos | Queso (mg/Kg) | Carne de res (mg/lb) | Requerimientos diarios mínimos (mg) |
|-----------------------------|---------------|----------------------|-------------------------------------|
| Vitamina A                  | 2             | 3                    | 2                                   |
| Vitamina B                  | 200           | 20                   | 60                                  |
| Vitamina C                  | 200           | 200                  | 25                                  |
| Colesterol                  | 40 Ud/Kg      | 100 Ud/lb            | -                                   |
| Costo                       | $2,500/Kg     | $2.750/lb            | -                                   |

**Plantear el anterior problema a través de un modelo de Programación Lineal (P.L.). (50 puntos)**

### Solución

#### Indentificación de tipo de problema
Indicadores de que es un problema de programación lineal:
* **¿Minimizar costo de dieta o minimizar consumo de sustancia?**: Sí, se busca minimizar el consumo de colesterol en la dieta
* **¿Requrimientos mínimos de nutrientes?**: Sí, se busca satisfacer los requerimientos mínimos de vitaminas.
* **¿Presupuesto limitado?**: Sí, se tiene un presupuesto limitado de $2,450.

#### Parámetros
* $i=1$: Queso.
* $i=2$: Carne de res.
* $j=1$: Vitamina A.
* $j=2$: Vitamina B.
* $j=3$: Vitamina C.
* $p_i$: Costo de del producto $i$.
* $a_i$: Cantidad de vitamina A en el alimento $i$.
* $b_i$: Cantidad de vitamina B en el alimento $i$.
* $c_i$: Cantidad de vitamina C en el alimento $i$.
* $d_i$: Cantidad de colesterol en el alimento $i$.
* $r_j$: Requerimiento mínimo de la vitamina $j$.
* $m$: Presupuesto de Carlos.

#### Variables de decisión
* $x_1$: Kg de queso a consumir en la dieta.
* $x_2$: Lb de carne de res a consumir en la dieta.

$$
\mathbf{x} = \begin{bmatrix} x_1 \\ x_2 \end{bmatrix}
$$

#### Restricciones

1. $\sum_{i=1}^{2} p_i x_i \leq m$: Restricción de presupuesto.
2. $\sum_{i=1}^{2} a_i x_i \geq r_{1}$: Restricción de vitamina A.
3. $\sum_{i=1}^{2} b_i x_i \geq r_{2}$: Restricción de vitamina B.
4. $\sum_{i=1}^{2} c_i x_i \geq r_{3}$: Restricción de vitamina C.
5. $x_i \geq 0, i=1,2$: Restricción de no negatividad.

#### Función objetivo
Si $\sum_{i=1}^{2} d_i x_i$ es la cantidad de colesterol en la dieta, entonces la función objetivo es:
$$
\min \sum_{i=1}^{2} d_i x_i
$$

#### Expresion matricial del modelo
$$
\begin{align*}
\min & \begin{bmatrix} d_1 & d_2 \end{bmatrix} \begin{bmatrix} x_1 \\ x_2 \end{bmatrix} \\
\text{s.t.} & \begin{bmatrix} -p_1 & -p_2 \\ a_1 & a_2 \\ b_1 & b_2 \\ c_1 & c_2 \end{bmatrix} \begin{bmatrix} x_1 \\ x_2 \end{bmatrix} \geq \begin{bmatrix} -m \\ r_{1} \\ r_{2} \\ r_{3} \end{bmatrix} \\
& x_i \geq 0, i=1,2
\end{align*}
$$
Notese para que la restricción de presupuesto fuera compatible con la forma canónica de las restricciones se multiplicó por -1.

In [8]:
import gurobipy as gp
from gurobipy import GRB

# Crear nuevo modelo
m1 = gp.Model("Problema de dieta")
# Crear variables
i = ["Queso", "Carne de res"]
j = ["Vitamina A", "Vitamina B", "Vitamina C"]
p = [2500, 2750]
a = [2, 3]
b = [200, 20]
c = [200, 200]
d = [140, 100]
r = [2, 60, 25]
m = 2450
x = m1.addMVar((2), name='x')

# Añadir restricciones
m1.addConstr(gp.quicksum(a[i]*x[i] for i in range(2)) >= r[0], "Vitamina_A")
m1.addConstr(gp.quicksum(b[i]*x[i] for i in range(2)) >= r[1], "Vitamina_B")
m1.addConstr(gp.quicksum(c[i]*x[i] for i in range(2)) >= r[2], "Vitamina_C")
m1.addConstr(gp.quicksum(p[i]*x[i] for i in range(2)) <= m, "Presupuesto")

# Crear función objetivo
c = [d[0], d[1]]
m1.setObjective(gp.quicksum(c[i]*x[i] for i in range(2)), GRB.MINIMIZE)
m1.update()
m1.write('Examen1Punto1-1.lp')
print(m1)
# Optimizar el modelo
m1.optimize()

# Revisar el estado de la solución
print('Optimization ended with status %d' % m1.status)
if (m1.status != GRB.OPTIMAL):
    print('No se encontró solución óptima')
else:
    # Imprimir solución
    print(f'El menor costo de la dieta es de {m1.objVal:.2f} con la siguiente ingesta:')
    for v in m1.getVars():
        print(f'{v.varName} = {v.x}')
    print('Objetivo: %g' % m1.objVal)
    
    # Mostrar holgura o exceso
    print("\n# Holgura o exceso de las restricciones:")
    for c in m1.getConstrs():
        print(f'{c.ConstrName}: {c.Slack:.2f}')

    # Mostrar precios sombra
    print("\n# Precios sombra de las restricciones:")
    for c in m1.getConstrs():
        print(f'{c.ConstrName}: {c.Pi:.2f}')

    # Obtener rangos de lados derechos
    print("\n# Rangos permisibles de los lados derechos (RHS):")
    for c in m1.getConstrs():
        print(f'{c.ConstrName}: {c.SARHSLow:.2f} to {c.SARHSUp:.2f}')   

    # Obtener rangos de coeficientes de la función objetivo
    print("\n# Rangos permisibles de los coeficientes de la función objetivo:")
    for v in m1.getVars():
        print(f'{v.VarName}: {v.SAObjLow:.2f} to {v.SAObjUp:.2f}')
    

<gurobi.Model Continuous instance Problema de dieta: 4 constrs, 2 vars, No parameter changes>
Gurobi Optimizer version 11.0.3 build v11.0.3rc0 (win64 - Windows 10.0 (19045.2))

CPU model: Intel(R) Core(TM) i5-4200U CPU @ 1.60GHz, instruction set [SSE2|AVX|AVX2]
Thread count: 2 physical cores, 4 logical processors, using up to 4 threads

Optimize a model with 4 rows, 2 columns and 8 nonzeros
Model fingerprint: 0xb0d7aacd
Coefficient statistics:
  Matrix range     [2e+00, 3e+03]
  Objective range  [1e+02, 1e+02]
  Bounds range     [0e+00, 0e+00]
  RHS range        [2e+00, 2e+03]
Presolve time: 0.02s
Presolved: 4 rows, 2 columns, 8 nonzeros

Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    0.0000000e+00   5.325000e+01   0.000000e+00      0s
       2    8.5000000e+01   0.000000e+00   0.000000e+00      0s

Solved in 2 iterations and 0.03 seconds (0.00 work units)
Optimal objective  8.500000000e+01
Optimization ended with status 2
El menor costo de la dieta es de 8

In [9]:
import gurobipy as gp
from gurobipy import GRB

# Crear nuevo modelo
m1 = gp.Model("Problema de dieta")
# Crear variables
i = ["Queso", "Carne de res"]
j = ["Vitamina A", "Vitamina B", "Vitamina C"]
p = [2500, 2750]
a = [2, 3]
b = [200, 20]
c = [200, 200]
d = [140, 100]
r = [2, 60, 25]
m = 2450
x = m1.addMVar((2), name='x')

# Crear restricciones como matrix
A = [[-p[0], -p[1]],
     [a[0],  a[1]],
     [b[0],  b[1]],
     [c[0],  c[1]]]
# Crear lado derecho de restricciones
b = [-m, r[0], r[1], r[2]]

m1.addConstrs((gp.quicksum(A[i][j]*x[j] for j in range(2)) >= b[i] for i in range(4)), name='c')

# Crear función objetivo
c = [d[0], d[1]]
m1.setObjective(gp.quicksum(c[i]*x[i] for i in range(2)), GRB.MINIMIZE)
m1.update()
m1.write('Examen1Punto1-2.lp')
print(m1)
# Optimizar el modelo
m1.optimize()

# Revisar el estado de la solución
print('Optimization ended with status %d' % m1.status)
if (m1.status != GRB.OPTIMAL):
    print('No se encontró solución óptima')
else:
    # Imprimir solución
    print(f'El menor costo de la dieta es de {m1.objVal:.2f} con la siguiente ingesta:')
    for v in m1.getVars():
        print(f'{v.varName} = {v.x}')
    print('Objetivo: %g' % m1.objVal)
    
    # Mostrar holgura o exceso
    print("\n# Holgura o exceso de las restricciones:")
    for c in m1.getConstrs():
        print(f'{c.ConstrName}: {c.Slack:.2f}')

    # Mostrar precios sombra
    print("\n# Precios sombra de las restricciones:")
    for c in m1.getConstrs():
        print(f'{c.ConstrName}: {c.Pi:.2f}')

    # Obtener rangos de lados derechos
    print("\n# Rangos permisibles de los lados derechos (RHS):")
    for c in m1.getConstrs():
        print(f'{c.ConstrName}: {c.SARHSLow:.2f} to {c.SARHSUp:.2f}')   

    # Obtener rangos de coeficientes de la función objetivo
    print("\n# Rangos permisibles de los coeficientes de la función objetivo:")
    for v in m1.getVars():
        print(f'{v.VarName}: {v.SAObjLow:.2f} to {v.SAObjUp:.2f}')
    

<gurobi.Model Continuous instance Problema de dieta: 4 constrs, 2 vars, No parameter changes>
Gurobi Optimizer version 11.0.3 build v11.0.3rc0 (win64 - Windows 10.0 (19045.2))

CPU model: Intel(R) Core(TM) i5-4200U CPU @ 1.60GHz, instruction set [SSE2|AVX|AVX2]
Thread count: 2 physical cores, 4 logical processors, using up to 4 threads

Optimize a model with 4 rows, 2 columns and 8 nonzeros
Model fingerprint: 0xbf65e9f6
Coefficient statistics:
  Matrix range     [2e+00, 3e+03]
  Objective range  [1e+02, 1e+02]
  Bounds range     [0e+00, 0e+00]
  RHS range        [2e+00, 2e+03]
Presolve removed 1 rows and 0 columns
Presolve time: 0.02s
Presolved: 3 rows, 2 columns, 6 nonzeros

Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    4.1792000e+01   6.262548e+01   0.000000e+00      0s
       2    8.5000000e+01   0.000000e+00   0.000000e+00      0s

Solved in 2 iterations and 0.02 seconds (0.00 work units)
Optimal objective  8.500000000e+01
Optimization ended with statu

## 2. 

A continuación se muestra el modelo de programación lineal de un problema de optimización.

$$
\begin{align*}
\text{Maximizar } z &= 180000x_1 + 220000x_2 + 20000x_3 \\
\text{sujeto a:} \\
15x_1 + 10x_2 + 8x_3 &\leq 18789 \\
15x_1 + 15x_2 + 4x_3 &\leq 18123 \\
3x_1 + 4x_2 + 2x_3 &\leq 9789 \\
x_1 &\geq 923 \\
x_2 &\geq 223 \\
x_1, x_2, x_3 &\geq 0
\end{align*}
$$

El valor $z^*$ es 26185000 correspondiente a $x_1 = 923$, $x_2 = 223$ y $x_3 = 233.75$.

a) **Formule el problema dual del problema anterior. (10 puntos)**

b) **Encuentre los valores de las variables duales. (25 puntos)**

### Solución

#### Expresión matricial del modelo
Dado el primal
$$
\begin{aligned}
&\text{max} \quad z = \mathbf{c}^T \mathbf{x} \\
&\text{s. a.} \\
&\mathbf{A}\mathbf{x} \leq \mathbf{b} \\
&\mathbf{x} \geq 0
\end{aligned}
$$
Donde
$$
\mathbf{x} = \begin{bmatrix} x_1 \\ x_2 \\ x_3 \end{bmatrix}, \quad
\mathbf{A} = 
\begin{bmatrix} 
15 & 10 & 8 \\ 
15 & 15 & 4 \\ 
3  & 4  & 2 \\
-1 & 0  & 0 \\
0  & -1 & 0 \\
\end{bmatrix}, \quad
\mathbf{b} = \begin{bmatrix} 18789 \\ 18123 \\ 9789 \\ -923 \\ -223 \end{bmatrix}, \quad
\mathbf{c} = \begin{bmatrix} 180000 \\ 220000 \\ 20000 \end{bmatrix}, \quad
$$

*Note que para tener una forma canonica del modelo se cambió la dirección de las restricciones mayores o iguales a menores o iguales.*

la solución del problema primal es 
$$
\mathbf{x}^* = \begin{bmatrix} 923 \\ 223 \\ 233.75 \end{bmatrix}
$$
y el valor óptimo 
$$
z^* = \mathbf{c}^T \mathbf{x}^* = 26185000
$$.

Tenemos el problema dual
$$
\begin{aligned}
&\text{min} \quad w = \mathbf{b}^T \mathbf{y} \\
&\text{s. a.} \\
&\mathbf{A}^T\mathbf{y} \geq \mathbf{c} \\
&\mathbf{y} \geq 0
\end{aligned}
$$
Donde
$$
\mathbf{y} = \begin{bmatrix} y_1 \\ y_2 \\ y_3 \\ y_4 \\ y_5 \end{bmatrix}, \quad
\mathbf{A}^T =
\begin{bmatrix}
15 & 15 & 3 & -1 & 0 \\
10 & 15 & 4 & 0 & -1 \\
8  & 4  & 2 & 0 & 0
\end{bmatrix}, \quad
\mathbf{c} = \begin{bmatrix} 180000 \\ 220000 \\ 20000 \end{bmatrix}, \quad
\mathbf{b} = \begin{bmatrix} 18789 \\ 18123 \\ 9789 \\ -923 \\ -223 \end{bmatrix}
$$

In [32]:
"""
#### Expresión matricial del modelo
Dado el primal
$$
\begin{aligned}
&\text{max} \quad z = \mathbf{c}^T \mathbf{x} \\
&\text{s. a.} \\
&\mathbf{A}\mathbf{x} \leq \mathbf{b} \\
&\mathbf{x} \geq 0
\end{aligned}
$$
Donde
$$
\mathbf{x} = \begin{bmatrix} x_1 \\ x_2 \\ x_3 \end{bmatrix}, \quad
\mathbf{A} = 
\begin{bmatrix} 
15 & 10 & 8 \\ 
15 & 15 & 4 \\ 
3  & 4  & 2 \\
-1 & 0  & 0 \\
0  & -1 & 0 \\
\end{bmatrix}, \quad
\mathbf{b} = \begin{bmatrix} 18789 \\ 18123 \\ 9789 \\ -923 \\ -223 \end{bmatrix}, \quad
\mathbf{c} = \begin{bmatrix} 180000 \\ 220000 \\ 20000 \end{bmatrix}, \quad
$$

*Note que para tener una forma canonica del modelo se cambió la dirección de las restricciones mayores o iguales a menores o iguales.*

la solución del problema primal es 
$$
\mathbf{x}^* = \begin{bmatrix} 923 \\ 223 \\ 233.75 \end{bmatrix}
$$
y el valor óptimo 
$$
z^* = \mathbf{c}^T \mathbf{x}^* = 26185000
$$.

Tenemos el problema dual
$$
\begin{aligned}
&\text{min} \quad w = \mathbf{b}^T \mathbf{y} \\
&\text{s. a.} \\
&\mathbf{A}^T\mathbf{y} \geq \mathbf{c} \\
&\mathbf{y} \geq 0
\end{aligned}
$$
Donde
$$
\mathbf{y} = \begin{bmatrix} y_1 \\ y_2 \\ y_3 \\ y_4 \\ y_5 \end{bmatrix}, \quad
\mathbf{A}^T =
\begin{bmatrix}
15 & 15 & 3 & -1 & 0 \\
10 & 15 & 4 & 0 & -1 \\
8  & 4  & 2 & 0 & 0
\end{bmatrix}, \quad
\mathbf{c} = \begin{bmatrix} 180000 \\ 220000 \\ 20000 \end{bmatrix}, \quad
\mathbf{b} = \begin{bmatrix} 18789 \\ 18123 \\ 9789 \\ -923 \\ -223 \end{bmatrix}
$$
"""

import gurobipy as gp
from gurobipy import GRB
import numpy as np

# Crear el modelo
primal = gp.Model("Examen1Punto2Primal")
# Crear variables
x = primal.addMVar(3, lb=0, name='x')
# Crear restricciones
A = np.array([[15, 10, 8], 
              [15, 15, 4], 
              [3,  4,  2], 
              [-1, 0,  0], 
              [0, -1,  0]])
# Lado derecho de las restricciones
b = np.array([18789, 18123, 9789, -923, -223])
# Coeficientes de la función objetivo
c = np.array([180000, 220000, 20000])
# Crear restricciones
primal.addConstr(A @ x <= b, name='c')
# Crear función objetivo
primal.setObjective(c @ x, GRB.MAXIMIZE)
# Actualizar el modelo
primal.update()
# Escribir el modelo a un archivo
primal.write('Examen1Punto2Primal.lp')
# Optimizar el modelo
primal.optimize()

# Obtener y mostrar los resultados
z_bfs = primal.ObjVal
print("\n# Valores óptimos de las variables de decisión:")
for v in primal.getVars():
    print(f'{v.VarName}: {v.X:.2f}')
print(f'\nValor Óptimo del Objetivo: {z_bfs:.2f}')

# Listas para almacenar variables básicas y no básicas
basic_vars = []
non_basic_vars = []

for var in primal.getVars():
    if var.VBasis == 0: 
        basic_vars.append(var)
    else:
        non_basic_vars.append(var)

# Imprimir las variables básicas y no básicas
print("Variables Básicas (BV):")
for v in basic_vars:
    print(f"{v.VarName} = {v.X}")

print("Variables No Básicas (NBV):")
for v in non_basic_vars:
    print(f"{v.VarName} = {v.X}")

print("\n# Matriz de restricciones A:")
print(A)

# Identificamos las variables básicas y no básicas
basic_indices = []
non_basic_indices = []
for i, v in enumerate(primal.getVars()):
    if v.VarName in [v.VarName for v in basic_vars]:
        basic_indices.append(i)
        print(f'Variable básica: {v.VarName} en posición {i}')
    else:
        non_basic_indices.append(i)
        print(f'Variable no básica: {v.VarName} en posición {i}')
    
# Extraemos la submatriz básica \(\mathbf{B}\)
B = A[:, basic_indices]

if B.shape[0] != B.shape[1]:
    print("\nError: La matriz básica \(\mathbf{B}\) no es cuadrada.")
else:
    # Extraemos los coeficientes básicos \(\mathbf{c}_{BV}\)
    c_BV = np.array([v.Obj for v in basic_vars])

    # Calcular \(\mathbf{B}^{-1}\)
    B_inv = np.linalg.inv(B)
    print("\n# Valores de \(\mathbf{B}\):")
    print(B)

    print("\n# Valores de \(\mathbf{B}^{-1}\):")
    print(B_inv)

    # Calcular \(\mathbf{c}_{BV} \mathbf{B}^{-1}\)
    c_BV_B_inv = np.dot(c_BV, B_inv)
    print("\n# Valores de \(\mathbf{c}_{BV} \mathbf{B}^{-1}\):")
    print(c_BV_B_inv)
    
# Encontrar las restricciones que son activas
binding_constraints_indices = []
non_binding_constraints_indices = []
# Resolver cada restricción con los valores óptimos de las variables de decisión
for i, ct in enumerate(primal.getConstrs()):
    ct_slack = ct.getAttr(GRB.Attr.Slack)
    if ct_slack == 0:
        binding_constraints_indices.append(i)
    else:
        non_binding_constraints_indices.append(i)
print("Las siguientes restricciones son activas, por lo que su variable dual es diferente de cero:")
for i in binding_constraints_indices:
    print(f'Restricción {i+1}: {primal.getConstrs()[i].ConstrName}, entonces y_{i+1} es diferente de cero')

z_bfs

# Mostrar holgura o exceso
print("\n# Holgura o exceso de las restricciones:")
for ct in primal.getConstrs():
    print(f'{ct.ConstrName}: {ct.Slack:.2f}')

# Mostrar precios sombra
print("\n# Precios sombra de las restricciones:")
for ct in primal.getConstrs():
    print(f'{ct.ConstrName}: {ct.Pi:.2f}')
print("El precio sombra (shadow price) muestra el cambio en el valor de la función objetivo por cada unidad adicional de RHS.\n")

# ------------------------------
  
# Crear el modelo dual
dual = gp.Model("Examen1Punto2Dual")
# Crear variables
y = dual.addMVar(5, lb=0, name='y')
# Agrergar restricciones
dual.addConstr(A.T @ y >= c, name='c')
# Crear función objetivo
dual.setObjective(b @ y, GRB.MINIMIZE)
# Actualizar el modelo
dual.update()
# Escribir el modelo a un archivo
dual.write('Examen1Punto2Dual.lp')
# Optimizar el modelo
dual.optimize()

# Obtener y mostrar los resultados
print("\n# Valores óptimos de las variables de decisión:")
for v in dual.getVars():
    print(f'{v.VarName}: {v.X:.2f}')
print(f'\nValor Óptimo del Objetivo: {dual.ObjVal:.2f}')

# Listas para almacenar variables básicas y no básicas
basic_vars = []
non_basic_vars = []
for var in dual.getVars():
    if var.VBasis == 0: 
        basic_vars.append(var)
    else:
        non_basic_vars.append(var)

# Imprimir las variables básicas y no básicas
print("Variables Básicas (BV):")
for v in basic_vars:
    print(f"{v.VarName} = {v.X}")

print("Variables No Básicas (NBV):")
for v in non_basic_vars:
    print(f"{v.VarName} = {v.X}")

print("\n# Matriz de restricciones A:")
print(A)

# Identificamos las variables básicas y no básicas
basic_indices = []
non_basic_indices = []
for i, v in enumerate(primal.getVars()):
    if v.VarName in [v.VarName for v in basic_vars]:
        basic_indices.append(i)
        print(f'Variable básica: {v.VarName} en posición {i}')
    else:
        non_basic_indices.append(i)
        print(f'Variable no básica: {v.VarName} en posición {i}')



Gurobi Optimizer version 11.0.3 build v11.0.3rc0 (win64 - Windows 10.0 (19045.2))

CPU model: Intel(R) Core(TM) i5-4200U CPU @ 1.60GHz, instruction set [SSE2|AVX|AVX2]
Thread count: 2 physical cores, 4 logical processors, using up to 4 threads

Optimize a model with 5 rows, 3 columns and 11 nonzeros
Model fingerprint: 0x3d417978
Coefficient statistics:
  Matrix range     [1e+00, 2e+01]
  Objective range  [2e+04, 2e+05]
  Bounds range     [0e+00, 0e+00]
  RHS range        [2e+02, 2e+04]
Presolve removed 3 rows and 0 columns
Presolve time: 0.01s
Presolved: 2 rows, 3 columns, 6 nonzeros

Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    2.7490800e+08   2.025194e+02   0.000000e+00      0s
       3    2.2888400e+08   0.000000e+00   0.000000e+00      0s

Solved in 3 iterations and 0.02 seconds (0.00 work units)
Optimal objective  2.288840000e+08

# Valores óptimos de las variables de decisión:
x[0]: 923.00
x[1]: 285.20
x[2]: 0.00

Valor Óptimo del Objetivo: 22888400